In [ ]:
# How to use locally downloaded LLM models like Llama
# Source - https://medium.com/@ai-for-devs.com/how-to-use-autogen-with-any-open-source-llm-for-free-5607f2bad2f4
# Source - https://microsoft.github.io/autogen/0.2/docs/topics/non-openai-models/local-ollama/
# Source - https://learn.deeplearning.ai/courses/ai-agentic-design-patterns-with-autogen/lesson/4/reflection-and-blogpost-writing

In [ ]:
# This notebook shows how to use AutoGen with locally downloaded LLM models like Llama.
# It requires installing the Ollama server and running it locally.
# You can install Ollama by following the instructions at https://ollama.com/docs/installation.
# Once you have Ollama installed, you can run the following command to pull the Llama model:
# ollama pull llama3:27b

# Make sure to install autogen library first:
# pip install autogen

In [ ]:
# TO RUN MODELS FROM THE HUGGINGFACE MODEL HUB (restricted by free token limit)
# llm_config = {"model": "gpt-3.5-turbo"}


# TO RUN LOCALLY DOWNLOADED LLM MODELS LIKE LLAMA
llm_config = {
			"config_list": [
				{
					"model": "gemma3:27b",
					"api_type": "ollama",
					"stream": False,
					"client_host": "http://127.0.0.1:11434"
				}
			]
}

In [ ]:
task = """
        Without rewriting the sentences entirely, finetune the given academic article by making corrections to the existing sentences. 
        Use editing rules such as sentence variation, pacing, paragraph variation, chapter variation, dialog tags, adverbs in dialogs, show vs tell, character development, plot development, 
        tense consistency, passive indicators, cliches, redundancies, unnecessary words, POV shifts, power words, sentence starters, sentence structure, 
        dale chall readability, flesch-kincaid, linsear write, spache readability, lexical diversity, sentence complexity, sentence length, and more. Retain the original dialogs as is - do not remove any of the original dialogs. Retain the original plot and the style of writing.
        Make minimal changes to the original content and return the modified content.

        This is the original content:
        "The sun was setting over the horizon, casting a golden glow across the fields. The wind whispered through the trees, carrying with it the scent of blooming flowers. In the distance, a lone figure could be seen walking along the path, their silhouette framed by the fading light. As they approached, it became clear that they were not alone; a small dog trotted happily beside them, its tail wagging in delight."

       """

In [ ]:
import autogen

# Create a professional writer assistant agent
writer = autogen.AssistantAgent(
    name="Writer",
    system_message="""You are an expert writer of academic articles. You are especially known for academically rich and engaging content.
        You must polish your writing based on the feedback you receive and give a refined version. Only return your final work without additional comments.""",
    human_input_mode="NEVER",
    llm_config=llm_config,
)

In [ ]:
# Add reflection by creating an Editor agent 

editor = autogen.AssistantAgent(
    name="Editor",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    human_input_mode="NEVER",
    system_message="""You are an expert at English grammar and an expert editor of academic articles. You are an expert at analyzing and editing the technical aspects of the sentences in academic articles. 
                You analyze academic articles based on various rules including "
                sentence variation, pacing, paragraph variation, chapter variation, dialog tags, adverbs in dialogs, show vs tell, character development, plot development, 
                tense consistency, passive indicators, cliches, redundancies, unnecessary words, POV shifts, power words, sentence starters, sentence structure, 
                dale chall readability, flesch-kincaid, linsear write, spache readability, lexical diversity, sentence complexity, sentence length, word length, and more.
                You review the work of the writer and provide constructive 
                feedback to help improve the quality of the content and make the content more interesting and professional.  
                Without actually rewriting the original sentences entirely, you give suggestions to finetune those sentences to make them more engaging and interesting. 
                You polish the writer's original work without totally rewriting the original sentences, into a more engaging and interesting article.""",
)

In [ ]:

# Add nested chat by creating a Legal Reviewer agent and an Ethics Reviewer agent

legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    human_input_mode="NEVER",
    system_message='''You are a legal reviewer, known for 
        your ability to ensure that content in academic articles is not pirated 
        and is free from any potential copyright infringement or plagiarism. 
        Make sure your suggestion is concise (within 3 bullet points), 
        concrete and to the point. 
        Begin the review by stating your role.''',
)

ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    human_input_mode="NEVER",
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [ ]:
# Add a meta reviewer to aggregate the comments of the other reviewers

meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    human_input_mode="NEVER",
    system_message="You are a meta reviewer, you aggregate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

In [ ]:
# Orchestrate the nested chats

def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chat_queue = [
    {
    "recipient": legal_reviewer,
    "message": reflection_message, 
    "summary_method": "reflection_with_llm",
    "summary_args": {"summary_prompt" : 
    "Return review into as JSON object only:"
    "{'Reviewer': '', 'Review': ''}.",},
    "max_turns": 1},

    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     
    {"recipient": meta_reviewer, 
    "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
    "max_turns": 1},
]


In [ ]:
editor.register_nested_chats(
    review_chat_queue,
    trigger=writer,
)

In [ ]:
res = editor.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=3,
    summary_method="last_msg"
)

print (res.summary)